# Understand numpy indexing

In [ ]:
#%matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

### A few ways to get test numpy arrays

In [ ]:
np.arange(3), np.arange(4,8), np.arange(5,1,-2)

For experiments with multiplication, arrays of primes may be helpful:

In [ ]:
def arangep(n, starting_index=0):
    sympy.sieve.extend_to_no(starting_index + n)
    return np.array(sympy.sieve._list[starting_index:starting_index + n])

In [ ]:
arangep(5), arangep(4,2)

# Shapes and Indexing

Indexing [basics](https://numpy.org/devdocs/user/basics.indexing.html#basics-indexing) and [details](https://numpy.org/devdocs/reference/arrays.indexing.html#arrays-indexing)

In [ ]:
a = np.arange(2*3*4).reshape(2,3,4); print(a)

Indexing is row-major order (smallest-address-delta last) (C-style):

In [ ]:
a[0,0,1], a[0,1,0], a[1,0,0]

In [ ]:
a[0], a[0,0], a[0,0,0]

In [ ]:
a[0], a[0][0], a[0][0][0]

In [ ]:
a.flat[7:12]

### Multiplicative-type operations

In [ ]:
a = arangep(2)
b = arangep(2,2)
a,b

Binary scalar operations on vectors just map

In [ ]:
a+1, a*2, a+b, a*b, b/a, b%a

[`dot`](https://numpy.org/devdocs/reference/generated/numpy.dot.html) is "alternative matrix product with different broadcasting rules"

In [ ]:
a.dot(b), b.dot(a)

In [ ]:
m = arangep(4,4).reshape(2,2); m

Matrix dot vector produces vector of dot products of rows of the matrix with the vector:

In [ ]:
m.dot(a), a.dot(m[0]), a.dot(m[1]), m[0], m[1]

vector dot matrix produces vector of dot products of columns of matrix with the vector:

In [ ]:
a.dot(m), a.dot(m[:,0]), a.dot(m[:,1]), m[:,0], m[:,1]

`@` is infix [matrix multiplication](https://numpy.org/devdocs/reference/generated/numpy.matmul.html#numpy.matmul)

In [ ]:
a, m, m @ a, a @ m, m.T @ a

Right-multiplication by a matrix is equivalent to left-multiplication by its transpose:

In [ ]:
a @ m, m.T @ a, a @ m.T, m @ a

## Einstein summation notation

Numpy provides [Einstein summation](https://mathworld.wolfram.com/EinsteinSummation.html) operations with [einsum](https://numpy.org/devdocs/reference/generated/numpy.einsum.html)
1. Repeated indices are implicitly summed over.
1. Each index can appear at most twice in any term.
1. Each term must contain identical non-repeated indices.

In [ ]:
es = np.einsum

 $$a_{ik}a_{ij} \equiv \sum_{i} a_{ik}a_{ij}$$

$$M_{ij}v_j=\sum_{j}M_{ij}v_j$$

In [ ]:
es('ij,j', m, a), es('ij,i', m, a)

In [ ]:
es('j,ij', a, m), es('i,ij', a, m)

Scalar multiplication bei

In [ ]:
all(es('ij,j', m, a) == es('j,ij', a, m))

### Lorem Ipsum

In [ ]:
m2 = np.zeros((2,3), np.int); m2

In [ ]:
m2[1] = np.arange(3); m2

In [ ]:
m3 = arangep(8).reshape(4,2).T; m3

In [ ]:
m3[:,0]

In [ ]:
m @ m3[:,0]

In [ ]:
h = m @ m3; h

In [ ]:
b, b[...,np.newaxis]

In [ ]:
h + b[...,np.newaxis]

## Convenient representations

Suppose you have many __x__ to run through a net. What is the convenient representation?

Consider a two-input net, e.g. the XOR net. We want to vectorize the evaluation of the net, and its backprop. In the case of XOR the entire input domain is four vectors: { (0,0), (0,1), (1,0), (1,1) }:

In [ ]:
X = np.array([0,0, 0,1, 1,0, 1,1]).reshape(4,2); X

This is a convenient ordering for input, with each input vector contiguous in memory. But it's not in the form of column vectors for the classical left-multiplication by a transformation matrix to yield a column matrix product.

In [ ]:
m = np.arange(4).reshape(2,2) + 1; m

In [ ]:
m @ np.array([1, 2]).reshape(2,1)

We can transpose the input before left-multiplying ...

In [ ]:
m @ X.T

... and transpose it back:

In [ ]:
Y = (m @ X.T).T; Y

Or we can be less pedantic about expressing the matrix multiply:

In [ ]:
X @ m.T

In Einstein summation notation:

In [ ]:
es('ij,kj', X, m)

If we really require the matrix on the left, we can index thus:

In [ ]:
es('ij,kj->ki', m, X)

---
### What way is faster?

In [ ]:
timeit(X @ m.T)

In [ ]:
timeit(es('ij,kj->ki', m, X))

In [ ]:
tm = m.T

In [ ]:
timeit(X @ tm)

No surprise, fastest is to have the transposed matrix ready. No surprise that the Einstein summation is slower, as it requires formulating a loop from the string of indexes. But what if the input data is much larger? E.g.

In [ ]:
Xlarge = np.arange(2*10000).reshape(10000,2); Xlarge

In [ ]:
timeit(Xlarge @ tm)

In [ ]:
timeit(es('ij,kj->ki', m, Xlarge))

The parsing of the index string and formulating a plan is maybe 1.6 µs, but the loop is 

In [ ]:
(156 + 1.4 - 3.01)/94.2

64% slower.

---

Adding another vector to each result vector of the multiply:

In [ ]:
a, a + Y, Y + a

Applying a function to each result:

In [ ]:
relu = np.vectorize(lambda x: max(0,x))

Try it out:

In [ ]:
t = arangep(10).reshape(5,2) - 12; t

In [ ]:
relu(t)

---

In [ ]:
X @ m.T

In [ ]:
es('ij,kj', X, m)

In [ ]:
X, m

___

# Partials

## Preliminaries

A matrix __M__ left-multiplies a (column) vector __x__ to produce a (column) vector __y__:

$$ \mathbf{M} \mathbf{x} = \mathbf{y}$$

Using Einstein summation notation, the matrix multiply is

$$m_{ij}x_j\equiv\sum_{j}m_{ij}x_j$$

In that notation, our equation is:

$$ m_{ij}x_j = y_i $$

A `python` example:

In [ ]:
M = arangep(4).reshape(2,2)
x = arangep(2,4)
# x = np.arange(2)+1
M,x

In [ ]:
es = np.einsum

In [ ]:
y = es('ij,j', M, x)
y

`numpy` treats a 1-d array as a row or a column vector for the matrix multiplication operator `@`, depending on what side of the matrix it appears, so we can also say

In [ ]:
M @ x, all(M@x == y)

### Partial derivative of a matrix multiply

How does $y$ vary with $x$, with $M$ held constant? I.e. what is $\partial\mathbf{y}/\partial\mathbf{x}$?

"In general, the partial derivative of an [n-ary](http://en.wikipedia.org/wiki/Arity) function $f(x_1, \dots, x_n)$ in the direction $x_i$ at the point $(a_1, \dots, a_n)$ [is defined](https://en.wikipedia.org/w/index.php?title=Partial_derivative) to be:"

$$\frac{\partial f}{\partial x_i}(a_1, \ldots, a_n) = \lim_{h \to 0}\frac{f(a_1, \ldots, a_i+h,\ldots,a_n) - f(a_1,\ldots, a_i, \dots,a_n)}{h} \tag{2.1} \label{partial}$$

The matrix equation $\mathbf{M} \mathbf{x} = \mathbf{y}$ can be written as

$$\mathbf{M}\mathbf{x} = \mathbf{F}(\mathbf{x})=\sum_i f_i(x_1, x_2, \dots x_n) \mathbf{\hat{y}}_i=\mathbf{y} \tag{2.2} \label{mmul}$$

where

$$f_i(x_1, x_2, \dots x_n) = y_i \tag{2.3}$$

which, by definition of matrix multiplication, can be written

$$ f_i(x_1, x_2, \dots x_n) = \sum_{j=1}^{n} m_{ij}x_j \tag{2.4}$$

Substituting (2.4) into (2.1):

$$\frac{\partial f_i}{\partial x_j}(a_1, \ldots, a_n) = 
\lim_{h \to 0}\frac{
  \sum_{k=1}^{n} m_{ik}(a_k + \delta_{kj}h)
- \sum_{k=1}^{n} m_{ik}a_k}{h} = \lim_{h \to 0}\frac{m_{ij}h}{h} = m_{ij}
\tag{2.5}$$

Where $\delta_{ij}$ is the [Kronecker delta function](https://mathworld.wolfram.com/KroneckerDelta.html):

$$ \delta_{ij} =
    \begin{cases}
            1 &         \text{for } i=j,\\
            0 &         \text{for } i\neq j.
    \end{cases}
$$

Giving:

$$\frac{\partial f_i}{\partial x_j}(a_1, \ldots, a_n) = \lim_{h \to 0}\frac{\sum_{k=1}^{n} m_{ik}a_k + m_{ij}h - \sum_{k=1}^{n} m_{ik}a_k}{h} = \lim_{h \to 0}\frac{m_{ij}h}{h} = m_{ij} \tag{2.6}$$

In Einstein notation, $ f_i(x_1, x_2, \dots x_n) =  m_{ij}x_j = y_i $

Approximating numerically with our example:

In [ ]:
(M@x - M@(x-np.array([0.001, 0]))) / 0.001, (M@x - M@(x-np.array([0, 0.001]))) / 0.001

### Outer product

In [ ]:
a, b = arangep(2), arangep(3,2)
a, b

In [ ]:
es('i,j', a, b), es('j,i', a, b)

# END
---

In [ ]:
t =  np.array([0,0, 0,1, 1,0, 1,1]).reshape(4,2); t

In [ ]:
f = lambda a, b: 1 if (a > 0.5) ^ (b > 0.5) else 0

In [ ]:
f(1,0), f(1,1)

In [ ]:
f(t[1,0], t[1,1])

In [ ]:
[f(x[0], x[1]) for x in t]

In [ ]:
def exor(a, b):
    return 1 if (a > 0.5) ^ (b > 0.5) else 1

In [ ]:
#np.vectorize(exor, signature='(i)->()')(t)

In [ ]:
f2 = lambda v: 1 if (v[0] > 0.5) ^ (v[1] > 0.5) else 0

In [ ]:
#np.vectorize(f2)(t)

In [ ]:
np.vectorize(f2, signature='(i)->()')(t)

In [ ]:
[a for a in t[0]]

---

In [ ]:
a = np.arange(25).reshape(5,5)
b = np.arange(5)
c = np.arange(6).reshape(2,3)

In [ ]:
a,b,c

In [ ]:
np.einsum('ii', a)

In [ ]:
np.einsum('ii->i', a)

In [ ]:
np.trace(a)

In [ ]:
np.einsum('ji', a)

In [ ]:
np.einsum('ji,i', a, b)

In [ ]:
a.dot(b)

In [ ]:
a[:,0]

In [ ]:
a[:,0].dot(b)

In [ ]:
a[:,1]

In [ ]:
d = np.arange(125).reshape(5,5,5)

In [ ]:
np.einsum('iii', d)

In [ ]:
sum([d[i][i][i] for i in range(5)])

In [ ]:
np.einsum('iij',d)

In [ ]:
np.einsum('iiz', d)

In [ ]:
[sum([d[i][i][j] for i in range(5)]) for j in range(5)]

In [ ]:
sum(a[:])

In [ ]:
a[0]

In [ ]:
timeit(np.einsum('iii', d))

In [ ]:
es = np.einsum

In [ ]:
es('ijk,kji',d,d)

In [ ]:
timeit(es('iii', d))

In [ ]:
es('i,ij', b, a)

In [ ]:
es('ij', a)

In [ ]:
es('i', b)

In [ ]:
g = np.arange(4).reshape(2,2)

In [ ]:
g

In [ ]:
es('ij,jk',g,g)

In [ ]:
g@g

In [ ]:
g[:]

In [ ]:
h = np.arange(2); h

In [ ]:
h.dot(g)

In [ ]:
es('i,ij', h, g)

In [ ]:
g.dot(h)

In [ ]:
es('ji,i', g, h)

In [ ]:
es('ij,j', g, h)

In [ ]:
g[0,1]

In [ ]:
np.array(1)

In [ ]:
np.array([1])

In [ ]:
np.array(2)

In [ ]:
np.array([1,2])

In [ ]:
np.array([2])

In [ ]:
np.array(0)

In [ ]:
np.array(0).shape

In [ ]:
np.array([0]).shape

In [ ]:
np.array(0)+1

In [ ]:
np.array([0])+1

In [ ]:
np.array(3).dot(np.array(5))

In [ ]:
np.array([3]).dot(np.array(5))

In [ ]:
np.array([3]).dot(np.array([5]))

In [ ]:
es('i,i', np.array([3]), np.array([5]))

In [ ]:
#es('i,i', np.array(3), np.array(5))

In [ ]:
es('', np.array(3), np.array(5))

In [ ]:
int(np.array(3))

In [ ]:
int(np.array([3]))

In [ ]:
a = np.arange(4).reshape(2,2) + 1; print(a)

In [ ]:
b = np.arange(2) + 1; print(b)

In [ ]:
b.dot(a)

In [ ]:
es('ij, jk', a, np.array([[1,0],[1,0]]))

In [ ]:
a[:,0]

In [ ]:
a[:,1]

In [ ]:
sum(a[:,0])

In [ ]:
a.T

In [ ]:
es('...j->...', a)

In [ ]:
a,b

In [ ]:
a.dot(b), b.dot(a)

In [ ]:
b.shape

In [ ]:
c = b.reshape(2,1); c

In [ ]:
b.dot(c), b@c

In [ ]:
es('...i,i...', b, c)

In [ ]:
timeit(b.dot(c))

In [ ]:
timeit(b@c)

In [ ]:
timeit(es('...i,i...', b, c))

In [ ]:
timeit(es('i,i...', b, c))

In [ ]:
a,b,c

In [ ]:
a@b

In [ ]:
a@c

In [ ]:
b.shape, c.shape

In [ ]:
es('ij,j...', a,c)

In [ ]:
es('ij,j', a, b)

In [ ]:
es('ij,j...', a, b)

In [ ]:
es('ij,j->i', a, b)

In [ ]:
es('ij,j->j', a, b)

In [ ]:
es('ij,i...', a,c)

In [ ]:
es('ij,j...', a, a)

In [ ]:
es('...j,ij', a, a)

In [ ]:
Xd = np.array([0,0,1,0,0,1,1,1]).reshape(2,4); Xd

In [ ]:
a

In [ ]:
a@Xd

In [ ]:
Xd.reshape(4,2)

In [ ]:
t = np.arange(8).reshape(4,2); t

In [ ]:
a @ t.T

In [ ]:
t.T

In [ ]:
Ellipsis

In [ ]:
b

In [ ]:
b[:, np.newaxis]

In [ ]:
t

In [ ]:
t[np.newaxis]

In [ ]:
x = np.arange(3); x

In [ ]:
x[:,np.newaxis] + x[np.newaxis,:]

In [ ]:
x[:,np.newaxis] * x[np.newaxis,:]

In [ ]:
x[np.newaxis,:] * x[:,np.newaxis]

In [ ]:
x[:,np.newaxis], x[np.newaxis,:]

In [ ]:
t.dot(np.arange(2) + 1)

In [ ]:
(np.arange(2) + 1).dot(t)

In [ ]:
t,a

In [ ]:
t.dot(a)

In [ ]:
np.prime

In [ ]:
import sympy

In [ ]:
np.array(list(sympy.sieve.primerange(2000,2050)))

In [ ]:
pa = lambda n: np.array([sympy.prime(i+1) for i in range(n)])

In [ ]:
pa(50)

In [ ]:
tp = pa(1000)

In [ ]:
tp[-1]

In [ ]:
tp2 = pa(10000)

In [ ]:
np.sign(np.arange(5)-2)

In [ ]:
np.array([1]*2).shape

In [ ]:
np.ones(np.array([3,5]).shape)

In [ ]:
M=np.arange(4).reshape(2,2)
b=np.arange(2)+1
x=np.arange(2)+5

In [ ]:
M@x + b, (M@x) + b

In [ ]:
a, b = arangep(2), arangep(3,2)
a, b

In [ ]:
es('i,j', a, b)